In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dwh.d_vm_vmp_promotion 
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
AS
WITH 
vmp_promotion AS ( 
    SELECT 
        '2000' AS business_unit, 
        * EXCEPT(rn, proc_date) 
    FROM (
        SELECT 
            'Miền Bắc' AS mien, * EXCEPT(mien),
            ROW_NUMBER() OVER (PARTITION BY ten_chuong_trinh, ma_dat_hang, ngay_bat_dau ORDER BY proc_date DESC) AS rn
        FROM {catalog_name}.udp_wcm_silver_dim.mb_vmp_chuongtrinhkhuyemai
        WHERE proc_date = (SELECT MAX(proc_date) FROM {catalog_name}.udp_wcm_silver_dim.mb_vmp_chuongtrinhkhuyemai)
        UNION ALL
        SELECT 
            'Miền Nam' AS mien, * EXCEPT(mien),
            ROW_NUMBER() OVER (PARTITION BY ten_chuong_trinh, ma_dat_hang, ngay_bat_dau ORDER BY proc_date DESC) AS rn
        FROM {catalog_name}.udp_wcm_silver_dim.mn_vmp_chuongtrinhkhuyemai
        WHERE proc_date = (SELECT MAX(proc_date) FROM {catalog_name}.udp_wcm_silver_dim.mn_vmp_chuongtrinhkhuyemai)
    ) a 
    WHERE a.rn = 1
),
vm_promotion AS ( 
    SELECT 
        '1500' AS business_unit, 
        * EXCEPT(rn, proc_date) 
    FROM (
        SELECT 
            'Miền Bắc' AS mien, * EXCEPT(mien),
            ROW_NUMBER() OVER (PARTITION BY ten_chuong_trinh, ma_dat_hang, ngay_bat_dau ORDER BY proc_date DESC) AS rn
        FROM {catalog_name}.udp_wcm_silver_dim.mb_vm_chuongtrinhkhuyemai
        WHERE proc_date = (SELECT MAX(proc_date) FROM {catalog_name}.udp_wcm_silver_dim.mb_vm_chuongtrinhkhuyemai)
        UNION ALL
        SELECT 
            'Miền Nam' AS mien, * EXCEPT(mien),
            ROW_NUMBER() OVER (PARTITION BY ten_chuong_trinh, ma_dat_hang, ngay_bat_dau ORDER BY proc_date DESC) AS rn
        FROM {catalog_name}.udp_wcm_silver_dim.mn_vm_chuongtrinhkhuyemai
        WHERE proc_date = (SELECT MAX(proc_date) FROM {catalog_name}.udp_wcm_silver_dim.mn_vm_chuongtrinhkhuyemai)
    ) a 
    WHERE a.rn = 1
)
SELECT * FROM vmp_promotion
UNION ALL
SELECT * FROM vm_promotion
;
""")
